# Example to develop and debug your applications

An application is composed of `Task`s. `Cornserve` has provided some builtin `Task`s, and here is an example to create a VLM `Task`.

In [6]:
from cornserve.task.builtins.mllm import MLLMTask, Modality
mllm = MLLMTask(
    model_id="Qwen/Qwen2-VL-7B-Instruct",
    modalities=[Modality.IMAGE],
)

You can deploy this task to the cluster so that you can later invoke it. To deploy, you need to create a `CornserveClient`.

In [8]:
from cornserve.frontend import CornserveClient
cornserve = CornserveClient(url="your-endpoint:port")

Connected to Cornserve gateway at your-endpoint:port


Deploy may take a few minutes at first.

In [9]:
cornserve.deploy(mllm)

TaskResponse(status=200, content='Tasks declared used')

The builtin `MLLMTask` takes `MLLMInput`, so you can construct the input and try to invoke it.
Note the `multimodal_data` field is a list of tuples.

In [11]:
from cornserve.task.builtins.mllm import MLLMInput
mllm_input = MLLMInput(
    prompt="What is this image about?",
    multimodal_data=[("image", "https://picsum.photos/seed/random/200/300")]
)

In [12]:
await mllm(mllm_input)

INFO 2025-04-30 12:07:37,552 [cornserve.task.base:490] Dispatching tasks to http://your-endpoint:port/tasks/invoke


MLLMOutput(response='The image shows a dog wrapped in a blanket. The blanket appears to be plaid or checkered, with a mix of colors including beige, pink, and green. The dog is sitting on the ground, and the blanket is wrapped around its body, covering its head and part of its back. The background suggests an outdoor setting, possibly a forest or a park, with trees and greenery visible. The overall scene gives a cozy and warm impression, as if the dog is being kept warm in the cold weather.')

Now we can build more complex Tasks. For example:

In [14]:
from cornserve.task.base import Task, TaskInput, TaskOutput
class MyTaskInput(TaskInput):
    """Task input model."""

    first_prompt: str
    second_prompt: str
    multimodal_data: list[tuple[str, str]] = []

class MyTaskOutput(TaskOutput):
    """Task output model."""
    response: str

class MyTask(Task):
    def __init__(self):
        super().__init__()
        self.mllm = MLLMTask(
            model_id="Qwen/Qwen2-VL-7B-Instruct",
            modalities=[Modality.IMAGE],
        )

    def invoke(self, input: MyTaskInput) -> MyTaskOutput:
        """Invoke the task with the given input."""
        mllm_input = MLLMInput(
            prompt=input.first_prompt,
            multimodal_data=input.multimodal_data,
        )
        mllm_output = self.mllm.invoke(mllm_input)


        input = MLLMInput(
            prompt=input.second_prompt,
            multimodal_data=input.multimodal_data,
        )

        output = self.mllm.invoke(input)
        return MyTaskOutput(response=f"{mllm_output.response} \n-----\n{output.response}")

We can deploy it and test it. Note the deployment finishes instantly because `Cornserve` can reuse the `UnitTask`s in each `Task`.

In [16]:
my_task = MyTask()
cornserve.deploy(my_task)

TaskResponse(status=200, content='Tasks declared used')

In [18]:
my_input = MyTaskInput(
    first_prompt="What is this image about?",
    second_prompt="Write a haiku",
    multimodal_data=[("image", "https://picsum.photos/seed/picsum/200/300")],
)
result = await my_task(my_input)
print(result.response)

INFO 2025-04-30 13:22:47,485 [cornserve.task.base:490] Dispatching tasks to http://your-endpoint:port/tasks/invoke


The image depicts a serene and majestic mountain landscape during what appears to be either sunrise or sunset. The sky is painted with soft hues of pink and orange, transitioning into a lighter blue as it stretches upwards. The clouds are scattered, adding texture to the sky and casting gentle shadows on the mountain.

In the foreground, there is a vast expanse of snow-covered ground, indicating a high-altitude or alpine environment. The snow appears pristine and undisturbed, reflecting the light from the sky.

Dominating the background is a towering mountain peak, which is partially obscured by the snow. The peak rises sharply against the sky, its rugged and jagged edges adding to the dramatic and awe-inspiring nature of the scene. The mountain's surface is covered in snow, with some areas showing the underlying rock or ice, suggesting it is a high-altitude mountain.

The overall atmosphere of the image is tranquil and picturesque, capturing the beauty and grandeur of nature. The inte

We no longer need the `mllm`, so can `teardown` it in the cluster. Note this returns instantly because its `UnitTask` are also used by `my_task`!

In [19]:
cornserve.teardown(mllm)

TaskResponse(status=200, content='Tasks declared not used')

Now we can convert this `Task` to an `App`. `App`s are defined with entrypoint of `AppRequest` and exit of `AppResponse`. The `serve` funtion of the `App` will be invoked in `Cornserve` whenever it receives an `AppRequest`.

In [46]:
from pydantic import BaseModel, ConfigDict

class Request(BaseModel):
    """App request model.

    Attributes:
        prompt: The prompt to send to the LLM.
        multimodal_data: List of tuples (modality, data URL).
    """

    prompt: str
    multimodal_data: list[tuple[str, str]] = []

    model_config = ConfigDict(extra="forbid")

class Response(BaseModel):
    """App response model.

    Attributes:
        response: The response from the LLM.
    """

    response: str

async def serve(request: Request) -> Response:
    """Main serve function for the app."""
    task_input = MyTaskInput(
        first_prompt=request.prompt,
        second_prompt="Write a haiku about the image",
        multimodal_data=request.multimodal_data
    )
    task_output = await my_task(task_input)
    return Response(response=task_output.response)

We can test the app we just wrote.

In [52]:
req = Request(
    prompt="What's the name of the flower?",
    multimodal_data=[("image", "https://upload.wikimedia.org/wikipedia/commons/thumb/f/fb/Crab_apple_flower_4004.jpg/2880px-Crab_apple_flower_4004.jpg")]
)

In [53]:
res = await serve(req)
print(res.response)

INFO 2025-04-30 13:48:02,233 [cornserve.task.base:490] Dispatching tasks to http://your-endpoint:port/tasks/invoke


The flower in the picture is a Crab Apple (Malus). Crab apples are a type of apple tree that produces small, often tart fruits. They are known for their beautiful blossoms, which come in various colors including pink, white, and red. 
-----
Soft pink petals,
Gentle blooms in spring's embrace,
Nature's sweet embrace.


Disconnect will teardown all the tasks you deployed.

In [54]:
cornserve.close()

Closed connection to Cornserve gateway.
Closed keep-alive thread.
